In [65]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import pymzml
import csv
import os
import pandas as pd
import numpy as np
import math
from matplotlib import pyplot as plt
import re
import plotly.express as px
from collections import defaultdict

import plotly.io as pio
import json
import plotly.graph_objs as go
import matplotlib.colors as mcolors

import json
import ipywidgets as widgets
from IPython.display import display

import warnings
from IPython.display import display, Image, clear_output
import time
from NO_AVERAGE_SCRIPTS import full_parse
from NO_AVERAGE_SCRIPTS import filter_dataframe
from NO_AVERAGE_SCRIPTS import hex_to_rgba_hex
from NO_AVERAGE_SCRIPTS import json_to_string
from NO_AVERAGE_SCRIPTS import prep_edge_R

from NO_AVERAGE_SCRIPTS import make_pie_chart_no_replicates

from NO_AVERAGE_SCRIPTS import average_pie_chart_no_repeats

from NO_AVERAGE_SCRIPTS import make_bar_plot_comparisons





In [89]:
df_old = pd.read_csv("Old_Only_14_tolerance.csv")
df_new = pd.read_csv("New_Only_14_tolerance.csv")
df_common = pd.read_csv("Common_14_tolerance.csv")

In [90]:
list(df_new)

['Unnamed: 0', 'Class', 'Transition', 'Lipid']

In [91]:
def change_lipid_names(input_string):
    # Split the string into individual lipids
    lipid_names = input_string.split('|')

    # Function to change individual lipid name
    def change_lipid_name(lipid_name):
        lipid_name = lipid_name.strip()

        # Remove "Lyso_" and add "L" before class name
        if lipid_name.startswith("Lyso_"):
            lipid_name = "L" + lipid_name[5:]
#         print(lipid_name)
        # Find and replace 'o' or 'p' after any character with 'O-' or 'P-'
        if "o(" in lipid_name:
            lipid_name = lipid_name.replace("o(", "(O-")
        if "p(" in lipid_name:
            lipid_name = lipid_name.replace("p(", "(P-")        
        if "o (" in lipid_name:
            lipid_name = lipid_name.replace("o (", "(O-")
        if "p (" in lipid_name:
            lipid_name = lipid_name.replace("p (", "(P-")        

        # Remove any white spaces before or after '('
        lipid_name = lipid_name.replace(" (", "(").replace("( ", "(")

        return lipid_name

    # Change each lipid name
    lipid_names = [change_lipid_name(lipid_name) for lipid_name in lipid_names]

    # Join lipid names back into one string
    output_string = ' | '.join(lipid_names)

    return output_string

# Test the function
print(change_lipid_names("Lyso_PE(12:0) | PE (12:0)"))  # Should print "LPE(12:0)|PE(12:0)"
print(change_lipid_names("PCo(44:5)"))  # Should print "PC(O-44:5)"
print(change_lipid_names("Lyso_PG(17:1) | PGp (18:0)"))  # Should print "LPG(17:1)|PG(P-18:0)"


LPE(12:0) | PE(12:0)
PC(O-44:5)
LPG(17:1) | PG(P-18:0)


In [92]:
def change_TG_format(lipid_name):
    # Split the string by "_"
    parts = lipid_name.split('_')
    
    # Remove any white spaces from the parts
    parts = [part.replace(' ', '') for part in parts]

    # Change "TAG" to "TG" and add brackets
    parts[0] = parts[0].replace("TAG", "[TG")

    # Add closing bracket to the first part
    parts[0] = parts[0] + "]"

    # Join the parts back together with an underscore
    lipid_name = '_'.join(parts)

    return lipid_name

# Test the function
print(change_TG_format("TAG(48:3)_FA 18:0"))  # Should print "[TG(48:3)]_FA18:0"
print(change_TG_format("TAG(48:3)_FA 18:1"))  # Should print "[TG(48:3)]_FA18:1"

[TG(48:3)]_FA18:0
[TG(48:3)]_FA18:1


In [93]:
classes = ['PG', 'PI', 'PS', 'PC', 'PI',"PE"]

df_old.loc[df_old['Class'].isin(classes), 'Lipid'] = df_old.loc[df_old['Class'].isin(classes), 'Lipid'].apply(change_lipid_names)


# classes2 = ["TAG"]
# df_old.loc[df['Class'].isin(classes2), 'Lipid'] = df_old.loc[df['Class'].isin(classes2), 'Lipid'].apply(change_TG_format)
# print(df_old)

In [94]:
classes2 = ["TAG"]
df_old.loc[df_old['Class'].isin(classes2), 'Lipid'] = df_old.loc[df_old['Class'].isin(classes2), 'Lipid'].apply(change_TG_format)

print(df_old)

     Unnamed: 0 Class      Transition  \
0             3   CAR   374.2 -> 85.1   
1             4   CAR   384.3 -> 85.1   
2             5   CAR   388.3 -> 85.1   
3             9   CAR   402.3 -> 85.1   
4            10   CAR   412.3 -> 85.1   
..          ...   ...             ...   
798        1492   TAG  990.9 -> 693.9   
799        1493   TAG  992.9 -> 663.9   
800        1494   TAG  992.9 -> 671.9   
801        1495   TAG  992.9 -> 693.9   
802        1496   TAG  992.9 -> 695.9   

                                                 Lipid  
0                    O-(11-carboxyundecanoyl)carnitine  
1           (5Z,8Z)-3-hydroxytetradecadienoylcarnitine  
2    2-Hydroxymyristoylcarnitine, 3-hydroxytetradec...  
3                   O-(13-carboxytridecanoyl)carnitine  
4           (9Z,12Z)-3-hydroxyhexadecadienoylcarnitine  
..                                                 ...  
798                                  [TG(60:1)]_FA18:2  
799                                  [TG(60:0)]_FA2

In [105]:
print(df_old[df_old['Class'] == 'TAG']['Lipid'])

420    [TG(48:3)]_FA18:0
421    [TG(48:3)]_FA18:1
422    [TG(48:3)]_FA16:0
423    [TG(48:3)]_FA16:1
424    [TG(48:2)]_FA18:0
             ...        
798    [TG(60:1)]_FA18:2
799    [TG(60:0)]_FA20:0
800    [TG(60:0)]_FA20:4
801    [TG(60:0)]_FA18:1
802    [TG(60:0)]_FA18:2
Name: Lipid, Length: 383, dtype: object


In [96]:
list(df_new)

['Unnamed: 0', 'Class', 'Transition', 'Lipid']

In [102]:
def get_lipid_set(lipid_string):
    # Split the string into individual lipids and remove leading/trailing whitespaces
    lipid_list = [lipid.strip() for lipid in lipid_string.split('|')]
    # Return a set of lipids
    return set(lipid_list)

# Select specific classes
classes = ['PG', 'PI', 'PS', 'PC', 'PI',"PE"]
df_old_filtered = df_old[df_old['Class'].isin(classes)]
df_new_filtered = df_new[df_new['Class'].isin(classes)]

# Normalize Lipid column in both filtered dataframes
df_old_filtered['LipidSet'] = df_old_filtered['Lipid'].apply(get_lipid_set)
df_new_filtered['LipidSet'] = df_new_filtered['Lipid'].apply(get_lipid_set)

# Concatenate all 'LipidSet's in df_new_filtered into one set
df_new_all_lipids = set.union(*df_new_filtered['LipidSet'].tolist())

# Check if any lipid from each 'LipidSet' in df_old_filtered is in df_new_all_lipids
df_old_filtered['IsInNew'] = df_old_filtered['LipidSet'].apply(lambda x: any(lipid in df_new_all_lipids for lipid in x))

# Filter rows where 'IsInNew' is False
df_missing = df_old_filtered[~df_old_filtered['IsInNew']]

print(df_missing)

     Unnamed: 0 Class      Transition                  Lipid  \
73          227    PC  494.3 -> 184.1              LPC(16:1)   
74          238    PC  534.3 -> 184.1              LPC(19:2)   
75          305    PC  774.6 -> 184.1             PC(O-36:1)   
76          306    PC  776.5 -> 184.1               PC(36:7)   
77          319    PC  790.6 -> 184.1   PC(36:0); PC(P-38:6)   
..          ...   ...             ...                    ...   
415        1087    PS  848.6 -> 663.6   PS(P-42:6)|PS(40:0)    
416        1091    PS  854.6 -> 669.6             PS(O-42:4)   
417        1093    PS  856.6 -> 671.6             PS(P-42:2)   
418        1094    PS  858.5 -> 673.5               PS(42:9)   
419        1096    PS  860.5 -> 675.5               PS(42:8)   

                   LipidSet  IsInNew  
73              {LPC(16:1)}    False  
74              {LPC(19:2)}    False  
75             {PC(O-36:1)}    False  
76               {PC(36:7)}    False  
77   {PC(36:0); PC(P-38:6)}    False

In [103]:
len(df_missing)

179

In [104]:
df_missing.to_csv("Missing from New thats in old PC_PI_PS_PG_PE.csv")

In [110]:
def get_tag_info(tag_string):
    # Split TAG and FA
    tag_parts = tag_string.split('_')
    fa = tag_parts[1] if len(tag_parts) > 1 else None
    tags = tag_parts[0]
    
    # Find all TAGs within brackets and split by '|'
    tag_set = set(re.findall('\[(.*?)\]', tags)[0].split('|'))

    return (tag_set, fa)

# Select TAG class
classes = ['TAG']
df_old_filtered = df_old[df_old['Class'].isin(classes)].copy()
df_new_filtered = df_new[df_new['Class'].isin(classes)].copy()

# Apply get_tag_info function to Lipid column in both dataframes
df_old_filtered['TagInfo'] = df_old_filtered['Lipid'].apply(get_tag_info)
df_new_filtered['TagInfo'] = df_new_filtered['Lipid'].apply(get_tag_info)

missing_rows = []

# Loop through each row in df_old_filtered
for idx, row in df_old_filtered.iterrows():
    tag_set_old, fa_old = row['TagInfo']
    
    # Check each row in df_new_filtered
    for _, row_new in df_new_filtered.iterrows():
        tag_set_new, fa_new = row_new['TagInfo']
        
        # Check if any TAG from df_old is in df_new and if the FA is the same
        if any(tag in tag_set_new for tag in tag_set_old) and fa_old == fa_new:
            break
    else:
        missing_rows.append(row)

# Create a dataframe for the missing rows
df_missing = pd.DataFrame(missing_rows)

print(df_missing)


IndexError: list index out of range

In [107]:
df_missing.to_csv("Missing from New thats in old TG.csv")